In [6]:
import pickle
import os

import pandas as pd
import numpy as np

In [54]:
def load_pickles(pickles_dir):
    for filename in os.listdir(pickles_dir):
        with open(os.path.join(pickles_dir, filename), 'rb') as f:
            save_dict = pickle.load(f)
        yield save_dict, filename

def write_pickle(save_dict, pickle_path):
    with open(pickle_path, 'wb') as f:
        pickle.dump(save_dict, f)

def compare(arr1, arr2):
    left = list(np.copy(arr1))
    for elem in arr2:
        if elem not in left:
            print(f'B contains {elem} while A does not')
        else:
            left.remove(elem)
    for elem in left:
        print(f'A contains {elem} while B does not')

In [56]:
datasets = None
detectors = []

path = os.path.join('..', 'reports', 'experiment_pollution', 'extreme_1', 'evaluators')
save_dicts_1 = []
for save_dict, filename in load_pickles(path):
    assert datasets is None or np.array_equal(datasets, save_dict['datasets'])
    if datasets is None:
        datasets = save_dict['datasets']
        print('Datasets: ', '\n'.join(datasets))
    if not np.array_equal(detectors, save_dict['detectors']):
        print('Add detectors: ', save_dict['detectors'])
        detectors += save_dict['detectors']
    save_dict['_filename'] = os.path.join(filename)
    save_dicts_1.append(save_dict)

path = os.path.join('..', 'reports', 'experiment_pollution', 'extreme_1', 'additional_evaluators')
save_dicts_2 = []
for save_dict, _ in load_pickles(path):
    assert datasets is None or np.array_equal(datasets, save_dict['datasets']), compare(datasets, save_dict['datasets'])
    datasets = datasets or save_dict['datasets']
    if not all([det in detectors for det in save_dict['detectors']]):
        print('Add detectors: ', save_dict['detectors'])
        detectors += save_dict['detectors']
    save_dicts_2.append(save_dict)


# --- Merge results --- #

path = os.path.join('..', 'reports', 'experiment_pollution', 'extreme_1', 'evaluators_merged')
os.makedirs(path, exist_ok=True)
for dict1, dict2 in zip(save_dicts_1, save_dicts_2):
    # We don't need the results values so drop them
    dict1['results'] = None
    dict1['seed'] = None
    
    # Drop results of old algorithm
    dict1['benchmark_results'] = dict1['benchmark_results'][dict1['benchmark_results'].algorithm != 'LSTMED']
    
    dict1['detectors'].append('AutoEncoder')
    dict1['benchmark_results'] = dict1['benchmark_results'].append(dict2['benchmark_results'])
    
    file_path = os.path.join(path, dict1['_filename'])
    dict1['_filename'] = None
    write_pickle(dict1, file_path)

Datasets:  Syn Extreme Outliers (pol=0.0, anom=0.01)
Syn Extreme Outliers (pol=0.0, anom=0.023)
Syn Extreme Outliers (pol=0.0, anom=0.05)
Syn Extreme Outliers (pol=0.0, anom=0.2)
Syn Extreme Outliers (pol=0.0, anom=0.5)
Syn Extreme Outliers (pol=0.25, anom=0.01)
Syn Extreme Outliers (pol=0.25, anom=0.023)
Syn Extreme Outliers (pol=0.25, anom=0.05)
Syn Extreme Outliers (pol=0.25, anom=0.2)
Syn Extreme Outliers (pol=0.25, anom=0.5)
Syn Extreme Outliers (pol=0.5, anom=0.01)
Syn Extreme Outliers (pol=0.5, anom=0.023)
Syn Extreme Outliers (pol=0.5, anom=0.05)
Syn Extreme Outliers (pol=0.5, anom=0.2)
Syn Extreme Outliers (pol=0.5, anom=0.5)
Syn Extreme Outliers (pol=0.75, anom=0.01)
Syn Extreme Outliers (pol=0.75, anom=0.023)
Syn Extreme Outliers (pol=0.75, anom=0.05)
Syn Extreme Outliers (pol=0.75, anom=0.2)
Syn Extreme Outliers (pol=0.75, anom=0.5)
Syn Extreme Outliers (pol=1.0, anom=0.01)
Syn Extreme Outliers (pol=1.0, anom=0.023)
Syn Extreme Outliers (pol=1.0, anom=0.05)
Syn Extreme Outl

In [ ]:
### Filter LSTMED from first collection